In [11]:
import re
import pandas as pd
import requests

import time #modulo para controlar los tiempos y los sleep (evitar el error 429)

#DEFINO LOS HEADERS: 
headers = {
"X-RapidAPI-Key": "47608e078dmsha9428889aafcc0ap1afc77jsn6e04b24d521c",
"X-RapidAPI-Host": "moviesdatabase.p.rapidapi.com"
}

#DEFINO LOS GENEROS 
genres = ["Drama", "Comedy", "Action", "Fantasy", "Horror", "Mystery", "Romance", "Thriller"]

sleep_time = 5

#SACAR TODAS LAS URLS: 

for genre in genres:

    for year in range(1990, 1990): 

        next = f"/titles?year={year}&genre={genre}&page=1"
        while next:

            url = f"https://moviesdatabase.p.rapidapi.com{next}"

            response = requests.get(url, headers=headers)
            #SI LA RESPUESTA ES 200 ENTONCES COMIENZO A HACER TODO EL LOOP DE LINKS Y EXTRAER DATOS: 
            if response.status_code == 200:
                # Procesamos la respuesta da data aqui, lo convertimos a JSONy extraemos la información:
                datos = response.json()
                # aquí ya podemos sacar la próxima página
                next = datos["next"]
                lista_de_diccionarios = datos["results"]
                lista_peliculas = []

                for datos in lista_de_diccionarios: 
      
                    lista_consumible = []

                    tipo = datos["titleType"]["text"]
                    if tipo not in ["Movie", "Short"]:
                        print(f"Continue - type is {tipo}") # Siempre poner un print así me entero de que pasa
                        continue # Esto lo que hace es parar aquí el flujo y seguir con la siguiente iteración

                    # Aqui ya puedo continuar con el resto de la lógica y hacer un append directamente
                    lista_consumible.append(tipo)
                    lista_consumible.append(datos["titleText"]["text"])

                    #para que no me de error el NONE de aquellas pelis que no tengan mes o año. añado el check de: 
                    release_date = datos.get("releaseDate")
                    if release_date:
                        mes = release_date.get("month")
                        anio = release_date.get("year")
                    else:
                        mes = None
                        anio = None
                    lista_consumible.append(mes)
                    lista_consumible.append(anio)
                    lista_consumible.append(datos["id"])


                    lista_peliculas.append(tuple(lista_consumible))

                # CONFIRMAME QUE ESTA PASANDO ...
                # ¡Ojo! Actualizo este print porque ya no tenemos page, podemos usar next
                # print(f"Received data for {genre} in {year}, page {page}")
                print(f"Received data from {next}")
            # SI LA RESPUESTA NO ES 200. TELL ME SOMETHING I DONT KNOW
            # Si hago muchas peticiones me da un 429, aquí puedo  añadir un elif mirando si el status_code 
            # es 429 y entonces aplicamos el sleep
            elif response.status_code == 429:
                print("Too many requests. Sleeping...")

                print(response.text) 
                time.sleep(sleep_time)
                
            else: # Este else lo dejo por si me salta otro error. 
                # OJO! page ya no existe puedo usar next
                #añadido este IF para printear cual es el error 429 (too many requests)
                if "error" in response.text:
                    error_message = response.json().get("error")
                    print(f"Error Message: {error_message}")

            print("...")
        


In [12]:
lista_peliculas

[('Movie', 'Porte aperte', 3, 1990, 'tt0100389'),
 ('Movie', 'Caluga o Menta', 10, 1990, 'tt0099210'),
 ('Movie', 'Alcune signore per bene', None, None, 'tt0099011'),
 ('Movie', 'On the Block', 10, 1989, 'tt0100293'),
 ('Movie', 'La désenchantée', 10, 1990, 'tt0099406'),
 ('Movie', 'Internal Affairs', 4, 1990, 'tt0099850')]

In [13]:
df = pd.DataFrame(lista_peliculas)

In [14]:
df

,0,1,2,3,4
0,Movie,Porte aperte,3.0,1990.0,tt0100389
1,Movie,Caluga o Menta,10.0,1990.0,tt0099210
2,Movie,Alcune signore per bene,NaN,NaN,tt0099011
3,Movie,On the Block,10.0,1989.0,tt0100293
4,Movie,La désenchantée,10.0,1990.0,tt0099406
5,Movie,Internal Affairs,4.0,1990.0,tt0099850
